# Рекомендация тарифов

# Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Значение `accuracy` должно быть не менее 0.75.

# Цель и задачи проекта

Целью настоящей работы была разработка модели для выбора подходящего тарифа для пользователя.

Для достижения цели в работе решались следующие задачи:
* Разделение данных на обучающую, валидационную и тестовую выборки;
* Обучение и сравнение моделей дерева решений, случайного леса и логистической регрессии по параметру `accuracy` на валидационной выборке;
* Проверка точности лучшей модели на тестовой выборке;
* Проверка лучшей модели на адекватность.

# Исходные данные

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Знакомство с данными

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.dummy import DummyClassifier

Создадим экземпляр класса pd.DataFrame() и поместим туда исследуемые данные. Далее посмотрим общую информацию.

In [2]:
try:
    data = pd.read_csv('users_behavior.csv')
except:
    data = pd.read_csv('https://raw.githubusercontent.com/PetrusPrimus-lab/Data_analysis_projects/refs/heads/main/Yandex_Practicum_Projects/tariffs_recommendation/users_behavior.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Имеем дело с таблицей из 3214 строк и 5 столбцов. В датафрейме отсутствуют пропуски. Названия столбцов приведены в змеином регистре. Проверим наличие явных дубликатов.

In [3]:
data.duplicated().sum()

0

Явные дубликаты отсутствуют. Все столбцы имеют числовой формат, поэтому вряд ли есть еще неявные дубликаты.

Изменим тип данных в столбцах `calls` и `messages` на `int64`.

In [4]:
data['calls'], data['messages'] = data['calls'].astype('int64'), data['messages'].astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


Посмотрим на распределения признаков.

In [5]:
px.histogram(
    data,
    'calls',
    nbins=30,
    title='Распределение количества звонков',
    labels={
        'calls': 'Количество звонков'
    }
)

In [6]:
px.histogram(
    data,
    'minutes',
    nbins=30,
    title='Распределение суммарной длительности звонков',
    labels={
        'minutes': 'Cуммарная длительность звонков, мин'
    }
)

In [7]:
px.histogram(
    data,
    'messages',
    nbins=30,
    title='Распределение количества sms-сообщений',
    labels={
        'messages': 'Количество sms-сообщений'
    }
)

In [8]:
px.histogram(
    data,
    'mb_used',
    nbins=30,
    title='Распределение израсходованного интернет-трафика',
    labels={
        'mb_used': 'Израсходованный интернет-трафик, Мб'
    }
)

Можно сделать следующие промежуточные выводы:
* На распределении количества звонков видны значения больше 190, кажущиеся выбросами. Не будем их удалять, не заручившись экспертным мнением, т.к. такое количество звонков могло быть. 
* Распределение суммарной длительности звонков демонстрирует незначительное количество очень долгих звонков: дольше 1200 мин. Такие случаи, действительно, могли быть. Оставим эти значения.
* Распределение количества sms-сообщений напоминает правоскошенное Пуассоновское. Преобладают клиенты, не пользующиеся sms-сообщениями. Также есть и любители переписки: более 150 sms-сообщений. В данном случае все значения, находящиеся после отметки в 150 sms, являются выбросами. Но это только в данной выборке. Такие клиенты в принципе могут быть. Не будем удалять эти значения.
* Распределение израсходованного интернет-трафика напоминает нормальное. Явных выбросов нет.

Таким образом, данные не нуждаются в дополнительной предобработке. На данном этапе только произведено изменеение типа данных в столбцах `calls` и `messages` с `float64` на `int64`. Перейдем к следующему этапу.

## Разбивка данных на выборки

Целью этого исследования является создание рекомендательной модели тарифов, поэтому целевым признаком будут значения столбца `is_ultra` (0 - тариф "Смарт", 1 - тариф "Ультра").

In [9]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

Разделим данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1 или 60%:20%:20%.

Первоначально выделим обучающую выборку и отделим ее от валидационной и тестовой.

In [10]:
features_train, features_other, target_train, target_other = (
    train_test_split(features, target, test_size=0.4, random_state=12345)
)

Теперь выделим валидационную и тестовую выборки.

In [11]:
features_valid, features_test, target_valid, target_test = (
    train_test_split(features_other, target_other, test_size=0.5, random_state=12345)
)

Посмотрим на размеры полученных выборок

In [12]:
print(f'Размер признаков и целевого признака обучающей выборки равны, соответственно: \
{features_train.shape} и {target_train.shape}')
print(f'Размер признаков и целевого признака валидационной выборки равны, соответственно: \
{features_valid.shape} и {target_valid.shape}')
print(f'Размер признаков и целевого признака тестовой выборки равны, соответственно: \
{features_test.shape} и {target_test.shape}')

Размер признаков и целевого признака обучающей выборки равны, соответственно: (1928, 4) и (1928,)
Размер признаков и целевого признака валидационной выборки равны, соответственно: (643, 4) и (643,)
Размер признаков и целевого признака тестовой выборки равны, соответственно: (643, 4) и (643,)


Данные разделены в заданном соотношении. Перейдем к следующему шагу.

## Исследование моделей

В этом разделе будем варьировать гиперпараметры разных алгоритмов обучения модели, чтобы подобрать такие значения, при которых параметр `accuracy` будет наибольшим. Для оценки временных затрат работы алгоритма используем класс `tqdm` из библиотеки `tqdm`. 

### Дерево решений

In [13]:
best_model_tree = None
best_depth_tree = 0
best_res_tree = 0
for i in tqdm(range(1, 11)):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model_tree.fit(features_train, target_train)
    accuracy_tree = model_tree.score(features_valid, target_valid)
    if accuracy_tree > best_res_tree:
        best_res_tree = accuracy_tree
        best_model_tree = model_tree
        best_depth_tree = i
tqdm.write(f'Лучшая точность модели дерева решений, {best_res_tree}, достигается при глубине дерева, равной {best_depth_tree}')

100%|██████████| 10/10 [00:00<00:00, 181.46it/s]

Лучшая точность модели дерева решений, 0.7853810264385692, достигается при глубине дерева, равной 3


### Случайный лес решений

In [14]:
best_model_rand_forest = None
best_estimators_rand_forest = 0
best_depth_rand_forest = 0
best_res_rand_forest = 0

for i in tqdm(range(1, 11)):
    for j in range(1, 11):
        model_rand_forest = RandomForestClassifier(random_state=12345, n_estimators=i, max_depth=j)
        model_rand_forest.fit(features_train, target_train)
        accuracy_rand_forest = model_rand_forest.score(features_valid, target_valid) 
        if accuracy_rand_forest > best_res_rand_forest:
            best_res_rand_forest = accuracy_rand_forest
            best_model_rand_forest = model_rand_forest
            best_estimators_rand_forest = i
            best_depth_rand_forest = j
tqdm.write(
    f'''
    Лучшая точность модели случайного леса, {best_res_rand_forest}, 
    достигается при количестве деревьев, равном {best_estimators_rand_forest}, и глубине дерева, равной {best_depth_rand_forest}
    '''
)

100%|██████████| 10/10 [00:01<00:00,  7.41it/s]


    Лучшая точность модели случайного леса, 0.80248833592535, 
    достигается при количестве деревьев, равном 8, и глубине дерева, равной 8
    


### Логистическая регрессия

In [15]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_log.fit(features_train, target_train)
res_log = model_log.score(features_valid, target_valid)
print(f'Точность модели логистической регрессии составляет {res_log}')

Точность модели логистической регрессии составляет 0.7558320373250389


**Вывод**

Ряд алгоритмов обучения моделей по точности выглядит следующим образом:
* Случайный лес решений: точность составляет `0.80248833592535` при 8 деревьях и глубине одного дерева, равной 8. Скорость составляет 5.41 ит./с
* Дерево решений: точность составляет `0.7853810264385692` при глубине дерева, равной 3. Скорость составляет 132.34 ит./с
* Логистическая регрессия: точность составляет `0.7107309486780715`

Для дальнейшего тестирования выбираем модель, обученную по алгоритму случайного леса.

## Проверка лучшей модели на тестовой выборке

In [16]:
accuracy_test = best_model_rand_forest.score(features_test, target_test) 
print(f'Точность модели случайного леса на тестовой выборке составляет {accuracy_test}')

Точность модели случайного леса на тестовой выборке составляет 0.7962674961119751


## Проверка лучшей модели на адекватность

Для проверки адекватности используем класс `DummyClassifier()`. Данный класс позволяет предсказать значение целевого признака, не опираясь на зависимости между целевым признаком и другими признаками. В этом случае предсказания делаются только на основе массива значений целевого признака, значения других признаков игнорируются. Если полученная точность модели `Dummy` будет ниже нашей лучшей модели случайного леса, то нашу модель можно считать адекватной.

In [17]:
model_dummy = DummyClassifier(random_state=12345)
model_dummy.fit(features_train, target_train)
dummy_score = model_dummy.score(features_valid, target_valid)

if dummy_score < best_res_rand_forest:
    print(f'''Точность модели dummy равна {dummy_score}, что меньше, чем у модели случайного леса ({best_res_rand_forest}).
    Наша модель адекватна''')
elif dummy_score > best_res_rand_forest:
    print(f'''Точность модели dummy равна {dummy_score}, что больше, чем у модели случайного леса ({best_res_rand_forest}).
    Наша модель неадекватна''')

Точность модели dummy равна 0.7060653188180405, что меньше, чем у модели случайного леса (0.80248833592535).
    Наша модель адекватна


In [18]:
#еще вариант посчитать accuracy константной модели
result = target_valid.value_counts(normalize=True)[0]
print(result)
print('Значения равны') if result==dummy_score else print('Значения не равны')

0.7060653188180405
Значения равны


## Выводы

В результате выполнения настоящей работы было обучено и сопоставлено по точности 3 модели классификации:
* Случайный лес решений: точность составляет `0.80248833592535` при 8 деревьях и глубине одного дерева, равной 8. Скорость составляет 5.41 ит./с
* Дерево решений: точность составляет `0.7853810264385692` при глубине дерева, равной 3. Скорость составляет 132.34 ит./с
* Логистическая регрессия: точность составляет `0.7107309486780715`

Лучший результат по точности показала модель случайного леса. Она оказалась не только точной, но и адекватной. Точность модели случайного леса на тестовой выборке составляет `0.7962674961119751`. Для создания рекомендательной системы тарифов следует использовать именно эту модель.